####import libraries

In [ ]:
import os
import glob
import cv2
import numpy as np
import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

####download,unzip dataset

**please insert USERNAME & PASSWORD to download dataset**

In [ ]:
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=USERNAME&password=PASSWORD&submit=Login' https://www.cityscapes-dataset.com/login/

#Download leftImg8bit_trainvaltest.zip
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=3

#gtFine_trainvaltest.zip
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1

!unzip "/content/leftImg8bit_trainvaltest.zip" -d "/content/Cityscape/"
!rm -rf /content/Cityscape/README
!rm -rf /content/Cityscape/license.txt

!unzip "/content/gtFine_trainvaltest.zip" -d "/content/Cityscape/"
!rm -rf /content/Cityscape/README
!rm -rf /content/Cityscape/license.txt

!rm -rf /content/sample_data
!rm -rf /content/cookies.txt
!rm -rf /content/index.html
#!rm -rf /content/leftImg8bit_trainvaltest.zip
#!rm -rf /content/gtFine_trainvaltest.zip

####delete useless files

In [ ]:
cityscapesPath = '/content/Cityscape'
searchFine1   = os.path.join( cityscapesPath , "gtFine"   , "*" , "*" , "*_gt*_polygons.json" )
searchFine2   = os.path.join( cityscapesPath , "gtFine"   , "*" , "*" , "*_gt*_instanceIds.png" )
searchFine3   = os.path.join( cityscapesPath , "gtFine"   , "*" , "*" , "*_gt*_color.png" )
#searchFine4   = os.path.join( cityscapesPath , "gtFine"   , "*" , "*" , "*_gt*_labelIds.png" )

filesFine1 = glob.glob( searchFine1 )
filesFine2 = glob.glob( searchFine2 )
filesFine3 = glob.glob( searchFine3 )
#filesFine4 = glob.glob( searchFine4 )
filesFine = filesFine1 + filesFine2 + filesFine3 #+ filesFine4
filesFine.sort()
#print(len(filesFine))

for item in filesFine:
    if item.endswith("color.png"):
        os.remove(item)
    elif item.endswith("instanceIds.png"):
        os.remove(item)
    #elif item.endswith("labelIds.png"):
    #    os.remove(item)
    elif item.endswith("polygons.json"):
        os.remove(item)

####read img and labels

In [ ]:
IMAGE_h = 256
IMAGE_w = 256
IMAGE_SIZE = 256
BATCH_SIZE = 4
NUM_CLASSES = 35
DATA_DIR = "/content/Cityscape"
NUM_TRAIN_IMAGES = 2975
NUM_VAL_IMAGES = 500

train_images = sorted(glob.glob(os.path.join(DATA_DIR, "leftImg8bit/train/*/*.png")))
train_masks = sorted(glob.glob(os.path.join(DATA_DIR, "gtFine/train/*/*.png")))
val_images = sorted(glob.glob(os.path.join(DATA_DIR, "leftImg8bit/val/*/*.png")))
val_masks = sorted(glob.glob(os.path.join(DATA_DIR, "gtFine/val/*/*.png")))

In [ ]:
print(len(train_images))
print(len(train_masks))
print(len(val_images))
print(len(val_masks))

2975
2975
500
500


In [ ]:
def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.image.decode_png(image, channels=1)
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMAGE_h, IMAGE_w])
    else:
        image = tf.image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_h, IMAGE_w])
        image = tf.keras.applications.resnet50.preprocess_input(image)
    return image


def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask


def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset


train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)


####define network

In [ ]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output


In [ ]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
#model.summary()


####compile and train

In [ ]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=loss,
    metrics=["accuracy"]
)

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=25)

####save and Load

In [ ]:
model.save("model.h5")

In [ ]:
filepath = '/content/model.h5'
model1 = tf.keras.models.load_model(
    filepath, custom_objects=None, compile=True, options=None
    )

####Drawing curves

In [ ]:
plt.plot(history.history["loss"])
plt.title("Training Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["accuracy"])
plt.title("Training Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_loss"])
plt.title("Validation Loss")
plt.ylabel("val_loss")
plt.xlabel("epoch")
plt.show()

plt.plot(history.history["val_accuracy"])
plt.title("Validation Accuracy")
plt.ylabel("val_accuracy")
plt.xlabel("epoch")
plt.show()


####predictions

In [ ]:
# Loading the Colormap

import numpy as np
colormap = [
    [128,  64, 128], [244,  35, 232], [70,  70,  70 ], [102, 102, 156], [190, 153, 153],
    [153, 153, 153], [250, 170,  30], [220, 220,   0], [107, 142,  35], [152, 251, 152],
    [ 70, 130, 180], [220,  20,  60], [255,   0,   0], [0,   0, 142  ], [0,   0,  70  ],
    [0,  60, 100  ], [0,  80, 100  ], [0,   0, 230  ], [119,  11,  32], [128,  64, 128],
    [244,  35, 232], [70,  70,  70 ], [102, 102, 156], [190, 153, 153], [153, 153, 153],
    [250, 170,  30], [220, 220,   0], [107, 142,  35], [152, 251, 152], [70, 130, 180 ],
    [220,  20,  60], [255,   0,   0], [0,   0, 142  ], [0,   0,  70  ], [0,  60, 100  ]]

In [ ]:
def infer(model, image_tensor):
    predictions = model.predict(np.expand_dims((image_tensor), axis=0))
    predictions = np.squeeze(predictions)
    predictions = np.argmax(predictions, axis=2)
    return predictions


def decode_segmentation_masks(mask, colormap, n_classes):
    r = np.zeros_like(mask).astype(np.uint8)
    g = np.zeros_like(mask).astype(np.uint8)
    b = np.zeros_like(mask).astype(np.uint8)
    for l in range(0, n_classes):
        idx = mask == l
        colormp = colormap[l]
        r[idx] = colormp[0]
        g[idx] = colormp[1]
        b[idx] = colormp[2]
    rgb = np.stack([r, g, b], axis=2)
    return rgb


def get_overlay(image, colored_mask):
    image = tf.keras.utils.array_to_img(image)
    image = np.array(image).astype(np.uint8)
    overlay = cv2.addWeighted(image, 0.35, colored_mask, 0.65, 0)
    return overlay


def plot_samples_matplotlib(display_list, figsize=(5, 3)):
    _, axes = plt.subplots(nrows=1, ncols=len(display_list), figsize=figsize)
    for i in range(len(display_list)):
        if display_list[i].shape[-1] == 3:
            axes[i].imshow(tf.keras.utils.array_to_img(display_list[i]))
        else:
            axes[i].imshow(display_list[i])
    plt.show()


def plot_predictions(images_list, colormap, model):
    for image_file in images_list:
        image_tensor = read_image(image_file)
        prediction_mask = infer(image_tensor=image_tensor, model=model)
        prediction_colormap = decode_segmentation_masks(prediction_mask, colormap, 20)
        overlay = get_overlay(image_tensor, prediction_colormap)
        plot_samples_matplotlib(
            [image_tensor, overlay, prediction_colormap], figsize=(18, 14)
        )

In [ ]:
plot_predictions(train_images[:4], colormap, model=model)